In [64]:
import pandas as pd
import json

In [65]:
json_file_path = "/Users/toddhendricks/Downloads/acndata_sessions.json"

with open(json_file_path, 'r') as j:
     data = json.loads(j.read())

In [66]:
df = pd.json_normalize(data['_items'])

In [67]:
charging_slots = df.stationID.nunique()
customers = df.userID.nunique()
total_mwh_delivered = sum(df.kWhDelivered) / 1000

In [68]:
df.columns

Index(['_id', 'clusterID', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')

In [69]:
df.connectionTime = pd.to_datetime(df.connectionTime)
df.doneChargingTime = pd.to_datetime(df.doneChargingTime)
# air_quality["datetime"] = pd.to_datetime(air_quality["datetime"])

In [70]:
df.connectionTime = pd.to_datetime(df.connectionTime.dt.strftime('%m/%d/%Y %H:%M')) 
df.doneChargingTime = pd.to_datetime(df.doneChargingTime.dt.strftime('%m/%d/%Y %H:%M'))

In [75]:
df['charging_session_duration'] = pd.to_timedelta((df.doneChargingTime - df.connectionTime),unit='h')

In [78]:
df['charging_session_duration'].astype("timedelta64[s]"")

0      0 days 01:59:00
1      0 days 02:23:00
2      0 days 00:34:00
3      0 days 00:31:00
4      0 days 07:58:00
             ...      
2347               NaT
2348               NaT
2349               NaT
2350               NaT
2351               NaT
Name: charging_session_duration, Length: 2352, dtype: timedelta64[ns]